# 07 — Module 9: Daily report (end-to-end pipeline)

This notebook runs the full daily pipeline and exports a CSV.

Pipeline:
1) build eligible universe
2) rank top N
3) compute entry signals
4) compute position sizing
5) generate a final report table

In [1]:
# If running from repo root and editable install is not done:
# pip install -e ".[dev]"

import pandas as pd
pd.set_option("display.width", 140)
pd.set_option("display.max_columns", 50)

> Tip: many modules assume the benchmark **SPY** is present (for Relative Strength).
> When using real tickers, include SPY:
>
> `tickers = ["AAPL","MSFT","NVDA","SPY"]`

In [2]:
from swing_screener.data.market_data import fetch_ohlcv, MarketDataConfig
from swing_screener.reporting.report import build_daily_report, export_report_csv, today_actions, ReportConfig
from swing_screener.risk.position_sizing import RiskConfig

tickers = [
    "AAPL",
    "MSFT",
    "NVDA",
    "AMZN",
    "META",
    "GOOGL",
    "TSLA",
    "AMD",
    "NFLX",
    "AVGO",
    "INTC",
    "ORCL",
    "CRM",
    "QCOM",
    "ADBE",
    "CSCO",
    "SHOP",
    "UBER",
    "ABNB",
    "SPY",
]
ohlcv = fetch_ohlcv(tickers, MarketDataConfig(start="2022-01-01"))

cfg = ReportConfig(
    risk=RiskConfig(account_size=500, risk_pct=0.01, k_atr=2.0, max_position_pct=0.60),
    only_active_signals=False,
)

report = build_daily_report(ohlcv, cfg)
report.head(20)

,rank,score,last,atr14,atr_pct,mom_6m,mom_12m,rs_6m,trend_ok,dist_sma50_pct,dist_sma200_pct,signal,breakout_level,ma20_level,entry,stop,shares,position_value,realized_risk,suggested_order_type,suggested_order_price,suggested_validity,execution_note,order_price_band_low,order_price_band_high
INTC,1,1.0,47.290001,2.205714,4.664229,1.063264,1.469452,0.941571,True,22.857338,67.793513,breakout,45.549999,38.968,47.29,42.88,1,47.29,4.41,BUY_LIMIT,46.738572,DAY,Breakout already occurred. Do NOT use buy-stop...,46.187144,47.290001


In [3]:
print(today_actions(report, max_rows=5))

TODAY ACTIONS (tradable signals):
 - INTC: breakout | entry~47.29 | stop~42.88 | shares=1 | risk~4.41


In [4]:
export_path = export_report_csv(report, "out/daily_report.csv")
export_path

'out/daily_report.csv'


## Details: report merging and execution guidance
Report pipeline (`build_daily_report`):
- eligible universe -> ranking -> signals -> position sizing -> merged report.
- The report keeps ranked features and joins signal/plan columns when available.
- If a plan is not possible (e.g., too expensive), `signal` stays but plan fields are blank.
- `only_active_signals=True` filters to `both`, `breakout`, `pullback`.

Execution guidance (`add_execution_guidance`):
- Adds columns: `suggested_order_type`, `suggested_order_price`, `suggested_validity`, `execution_note`, `order_price_band_low/high`.
- Breakout not triggered -> BUY_STOP slightly above `breakout_level`.
- Breakout already triggered -> BUY_LIMIT pullback (if second-chance enabled).
- Pullback signal -> BUY_LIMIT near `ma20_level` with a small ATR band.



## Glossary (tickers and metrics)
- SPY: ETF tracking the S&P 500. We use it as the benchmark for Relative Strength (RS).
- SMA: Simple Moving Average. Used for trend filters and pullback signals.
- ATR: Average True Range. Measures typical daily range; used for stop distance and sizing.
- RS: Relative Strength = momentum vs benchmark (e.g., stock 6m return minus SPY 6m return).
- R (R-multiple): Risk unit where 1R = entry - stop. Used in backtests and trade management.
